In [1]:
import numpy  as np
import pandas as pd
from metrics import MSE
from tqdm import tqdm

In [2]:
X_train = pd.read_pickle("../../../../Datasets/temporal/final/X_train.pkl")
c = X_train.columns.tolist()
X_val = pd.read_pickle("../../../../Datasets/temporal/final/X_val.pkl")
X_test = pd.read_pickle("../../../../Datasets/temporal/final/X_test.pkl")
y_train = pd.read_pickle("../../../../Datasets/temporal/final/y_train.pkl").to_numpy()
y_val = pd.read_pickle("../../../../Datasets/temporal/final/y_val.pkl").to_numpy()
y_test = pd.read_pickle("../../../../Datasets/temporal/final/y_test.pkl").to_numpy()

In [3]:
X_train.insert(X_train.shape[1], "rating", y_train.T[0])

In [4]:
state_columns = [i for i in c if "state" in i]

In [5]:
mean_per_state_dict = {}
for state in state_columns:
    state_df = X_train.groupby([state, "gPlusPlaceId"]).agg({"gPlusUserId": "count"}).reset_index()
    popular_place = state_df[state_df[state] == 1].sort_values("gPlusUserId", ascending=False).iloc[0]["gPlusPlaceId"]
    mean_per_state_dict[state] = X_train[(X_train[state] == 1) & (X_train["gPlusPlaceId"] == popular_place)]["rating"].mean()

In [6]:
mean_per_state_dict

{'state_AL': 4.947368421052632,
 'state_AR': 4.135135135135135,
 'state_AZ': 4.473684210526316,
 'state_CA': 3.788732394366197,
 'state_CO': 3.2535211267605635,
 'state_CT': 3.875,
 'state_DE': 3.823529411764706,
 'state_GA': 3.2127659574468086,
 'state_IA': 4.285714285714286,
 'state_ID': 4.482758620689655,
 'state_IL': 4.5675675675675675,
 'state_IN': 4.260273972602739,
 'state_KS': 4.476190476190476,
 'state_KY': 4.5,
 'state_LA': 3.6666666666666665,
 'state_MA': 4.327868852459017,
 'state_MD': 3.9183673469387754,
 'state_ME': 4.111111111111111,
 'state_MI': 4.285714285714286,
 'state_MN': 3.782608695652174,
 'state_MO': 4.492537313432836,
 'state_MS': 4.1,
 'state_MT': 4.6923076923076925,
 'state_NC': 4.489795918367347,
 'state_ND': 3.4285714285714284,
 'state_NE': 3.4,
 'state_NH': 4.583333333333333,
 'state_NJ': 3.822429906542056,
 'state_NM': 3.6153846153846154,
 'state_NV': 4.25,
 'state_NY': 4.055555555555555,
 'state_OH': 4.973333333333334,
 'state_OK': 4.517241379310345,
 's

In [7]:
y_pred_val = []
for i in tqdm(range(len(X_val))):
    for j in range(len(X_val.iloc[i][state_columns])):
        if X_val.iloc[i][state_columns][j] == 1:
            y_pred_val.append(mean_per_state_dict[state_columns[j]])
            break

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 161526/161526 [11:24<00:00, 235.98it/s]


In [8]:
MSE(predictions=y_pred_val, labels=y_val)[0]

1.9861741253742273

In [9]:
y_pred = []
for i in tqdm(range(len(X_test))):
    for j in range(len(X_test.iloc[i][state_columns])):
        if X_test.iloc[i][state_columns][j] == 1:
            y_pred.append(mean_per_state_dict[state_columns[j]])
            break

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 161526/161526 [10:54<00:00, 246.68it/s]


In [10]:
MSE(predictions=y_pred, labels=y_test)[0]

2.8617612596606503